
# KNN-Explainer Example-Notebook

This Notebook shows the KNN-Explainer and how to use it,
and its three difrent KNN-Shapley Variants.

Imports:

In [1]:
from __future__ import annotations

import os
import sys

notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.insert(0, project_root)

In [2]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from shapiq import InteractionValues
from sklearn.model_selection import train_test_split
import pandas as pd

from shapiq_student.knn_explainer import KNNExplainer
from shapiq_student.knn_shapley import KNNShapley
from shapiq_student.threshold import Threshold
from shapiq_student.weighted import Weighted

Test/Tain split:

In [3]:
df = pd.read_csv("movie_dataset.csv")
train_df, test_df = train_test_split(df, test_size=0.4, random_state=42)
X_train = train_df[["fight_scenes","kiss_scenes"]].values
y_train = train_df["label"].values
X_test = test_df[["fight_scenes","kiss_scenes"]].values
y_test = test_df["label"].values

## Normal KNN-Shapley:

In [4]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [5]:
explainer = KNNExplainer(knn, X_train, y_train, 0)

In [6]:
phi = explainer.explain(x = X_test)

In [7]:
print(phi)

InteractionValues(
    index=SV, max_order=1, min_order=1, estimated=True, estimation_budget=None,
    n_players=18, baseline_value=0.0,
    Top 10 interactions:
        (1,): 0.08333333333333336
        (2,): 0.08333333333333336
        (4,): 0.08333333333333336
        (5,): 0.08333333333333336
        (6,): 0.08333333333333336
        (7,): 0.08333333333333336
        (9,): 0.08333333333333336
        (14,): 0.08333333333333336
        (15,): 0.08333333333333336
        (17,): -0.027777777777777787
)


## Threshold KNN-Shapley:

In [8]:
knnT = RadiusNeighborsClassifier(15.0)
knnT.fit(X_train, y_train)

RadiusNeighborsClassifier(radius=15.0)

In [9]:
explainerT = KNNExplainer(knnT, X_train, y_train, class_index=y_test)

c:\Users\schul\anaconda3\Lib\site-packages\shapiq\explainer\base.py:130: UserWarning: The `data` and the model must be compatible. The model's prediction must be a 1-dimensional array.
  validate_data_predict_function(data, predict_function=self.predict, raise_error=False)


In [10]:
phiT = explainerT.explain(x=X_test)

In [11]:
print(phiT)

InteractionValues(
    index=SV, max_order=1, min_order=1, estimated=True, estimation_budget=None,
    n_players=18, baseline_value=0.0,
    Top 10 interactions:
        (1,): -8.326672684688674e-17
        (2,): -8.326672684688674e-17
        (3,): -8.326672684688674e-17
        (4,): -8.326672684688674e-17
        (5,): -8.326672684688674e-17
        (6,): -8.326672684688674e-17
        (7,): -8.326672684688674e-17
        (8,): -8.326672684688674e-17
        (16,): -8.326672684688674e-17
        (17,): -8.326672684688674e-17
)


## Weighted KNN-Shapley:

In [12]:
knnW = KNeighborsClassifier(n_neighbors=5)
knnW.fit(X_train, y_train)

KNeighborsClassifier()

In [13]:
row = df[df['title'] == 'Kiss Me Again'].iloc[0]
x_val = [row['fight_scenes'], row['kiss_scenes']]
y_val = row['label']

In [14]:
explainerW = Weighted(X_train, y_train, "weighted")

In [15]:
phiW = explainerW.weighted_knn_shapley(
    x_val=x_val, y_val=y_val,
    gamma=5, K=5)

In [16]:
print(phiW)

[ 1.25503663  1.25503663  1.25503663  1.25503663  1.25503663  0.25503663
  0.01497253  0.03264652  0.05050366  0.05844017  0.06240842  0.06457293
  0.06583555 -0.00678488  0.          0.        ]
